* data_setup.py (create dataset and dataloader)
* model_builder.py (build model here)
* engine.py (train_step(), test_step(), train() all are here)
* utils.py (here save _model() function)
* train.py (this will allow to all the file and train model with a single line command)


In [1]:
import os
os.makedirs("going_modular", exist_ok = True)

In [2]:
%%writefile going_modular/get_data.py
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...") 
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Writing going_modular/get_data.py


In [3]:
!python going_modular/get_data.py

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


In [4]:
#turn above get data into a python script such as get_data.py
#just use magic command

In [ ]:
#to delete directory use following command
!rm -rf data/

In [5]:
%%writefile going_modular/data_setup.py
import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
NUM_WORKERS = os.cpu_count()
def create_dataloaders(
  train_dir: str,
  test_dir: str, 
  transform: transforms.Compose,
  batch_size: int,
  num_workers: int = NUM_WORKERS
):
  train_data = datasets.ImageFolder(train_dir, transform = transform)
  test_data = datasets.ImageFolder(test_dir, transform = transform)
  class_names = train_data.classes
  train_dataloader = DataLoader(
    train_data,
    batch_size = batch_size,
    shuffle = True,
    num_workers = num_workers,
    pin_memory=True,
  )
  test_dataloader = DataLoader(
    test_data,
    batch_size = batch_size,
    shuffle =False,
    num_workers = num_workers,
    pin_memory = True,
  )
  return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [6]:
%%writefile going_modular/model_builder.py
import torch
from torch import nn
class TinyVGG(nn.Module):
  def __init__(self, input_shape:int, hidden_units: int, output_shape:int):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels = input_shape,
                  out_channels = hidden_units,
                  kernel_size = 3,
                  stride =1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_units,
                  out_channels = hidden_units,
                  kernel_size =3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units *13*13,
                  out_features = output_shape)
    )
  def forward(self, x: torch.Tensor):
      #x = self.conv_block_1(x)
      #x = self.conv_block_2(x)
      #x = self.classifier(x)
      return self.classifier(self.conv_block_2(self.conv_block_1(x)))


Writing going_modular/model_builder.py


In [7]:
%%writefile going_modular/engine.py
from typing import Dict, List, Tuple
import torch 
from tqdm.auto import tqdm
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device):
  model.train()
  train_loss, train_acc = 0, 0
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item() /len(y_pred)
  train_loss = train_loss /len(dataloader)
  train_acc = train_acc /len(dataloader)
  return train_loss, train_acc
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device):
  model.eval()
  test_loss, test_acc = 0, 0
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      test_pred_logits = model(X)
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels ==y).sum().item() / len(test_pred_labels))
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc /len(dataloader)
  return test_loss, test_acc
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device):
  results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
  return results

Writing going_modular/engine.py


In [8]:
%%writefile going_modular/utils.py
from pathlib import Path
import torch
def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents = True,
                        exist_ok = True)
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path /model_name
  print(f"[info] saving model to : {model_save_path}")
  torch.save(obj=model.state_dict(),
             f =model_save_path)

Writing going_modular/utils.py


In [9]:
%%writefile going_modular/train.py
import os
import torch 
from torchvision import transforms
import data_setup, engine, model_builder, utils
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS =10
LEARNING_RATE = 0.001
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"
device = "cuda" if torch.cuda.is_available() else "cpu"
data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir, 
    test_dir = test_dir,
    transform = data_transform,
    batch_size = BATCH_SIZE
)
model = model_builder.TinyVGG(
    input_shape =3,
    hidden_units = HIDDEN_UNITS,
    output_shape = len(class_names)
).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
engine.train(model = model,
             train_dataloader = train_dataloader,
             test_dataloader = test_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             epochs = NUM_EPOCHS,
             device = device)
utils.save_model(model=model,
                 target_dir = "models",
                 model_name = "05_going_modular_script_mode_tinyvgg_model.pth")

Writing going_modular/train.py


In [10]:
!python going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1191 | train_acc: 0.2656 | test_loss: 1.1028 | test_acc: 0.2604
 20% 1/5 [00:08<00:34,  8.61s/it]Epoch: 2 | train_loss: 1.1109 | train_acc: 0.3203 | test_loss: 1.1192 | test_acc: 0.1979
 40% 2/5 [00:11<00:15,  5.00s/it]Epoch: 3 | train_loss: 1.0957 | train_acc: 0.2930 | test_loss: 1.0980 | test_acc: 0.1875
 60% 3/5 [00:12<00:06,  3.44s/it]Epoch: 4 | train_loss: 1.0926 | train_acc: 0.5000 | test_loss: 1.0877 | test_acc: 0.6146
 80% 4/5 [00:14<00:02,  2.63s/it]Epoch: 5 | train_loss: 1.0769 | train_acc: 0.5078 | test_loss: 1.0699 | test_acc: 0.5142
100% 5/5 [00:15<00:00,  3.09s/it]
[info] saving model to : models/05_going_modular_script_mode_tinyvgg_model.pth


**Now use argparse in train.py**

use python argparse module to be able to send the train.py custom hyperparameter values for training procedures.
like add an argument flag for using a different:
* Training/testing directory
* Learning rate
* batch size
* number of epochs to train for
* number of hidden units in the TinyVGG model
run like

!python train.py --batch_size 64 --learning_rate 0.001 

In [ ]:
#!python train.py --batch_size 64 --learning_rate 0.001 --num_epochs 25
#modify train.py for accep extra argument

In [11]:
%%writefile going_modular/train_arg.py
import os
import argparse
import torch 
from torchvision import transforms
import data_setup, engine, model_builder, utils

#create a parser
parser = argparse.ArgumentParser(description = "get some hyperparameters.")

#get an arg for num_epochs
parser.add_argument("--num_epochs",
                    default =10,
                    type=int,
                    help="the number of epochs to train for")
#get an arg for batch_size
parser.add_argument("--batch_size",
                    default = 32,
                    type = int,
                    help = "the number of sample per batch ")
#get an arg for hidden_units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help = " number of hidden units in hidden layers")
#get an arg for learning rate
parser.add_argument("--learning_rate",
                    default = 0.001,
                    type = float,
                    help= "learning rate to use for model")

#get out arguments from the parser
args = parser.parse_args()
#setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS =args.hidden_units
LEARNING_RATE = args.learning_rate
print(f"[info] training a model for {NUM_EPOCHS} epochs with a {BATCH_SIZE} batch_size using {HIDDEN_UNITS} hidden unist and a learning rate {LEARNING_RATE} ")
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"
device = "cuda" if torch.cuda.is_available() else "cpu"
data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir, 
    test_dir = test_dir,
    transform = data_transform,
    batch_size = BATCH_SIZE
)
model = model_builder.TinyVGG(
    input_shape =3,
    hidden_units = HIDDEN_UNITS,
    output_shape = len(class_names)
).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
engine.train(model = model,
             train_dataloader = train_dataloader,
             test_dataloader = test_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             epochs = NUM_EPOCHS,
             device = device)
utils.save_model(model=model,
                 target_dir = "models",
                 model_name = "05_going_modular_script_mode_tinyvgg_model.pth")

Writing going_modular/train_arg.py


In [12]:
!python going_modular/train_arg.py

[info] training a model for 10 epochs with a 32 batch_size using 10 hidden unist and a learning rate 0.001 
  0% 0/10 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.0971 | train_acc: 0.4336 | test_loss: 1.1054 | test_acc: 0.1979
 10% 1/10 [00:04<00:37,  4.20s/it]Epoch: 2 | train_loss: 1.0862 | train_acc: 0.4141 | test_loss: 1.1118 | test_acc: 0.1979
 20% 2/10 [00:05<00:20,  2.54s/it]Epoch: 3 | train_loss: 1.0885 | train_acc: 0.2930 | test_loss: 1.1134 | test_acc: 0.2292
 30% 3/10 [00:06<00:14,  2.01s/it]Epoch: 4 | train_loss: 1.0558 | train_acc: 0.4453 | test_loss: 1.0601 | test_acc: 0.4044
 40% 4/10 [00:08<00:10,  1.76s/it]Epoch: 5 | train_loss: 0.9385 | train_acc: 0.5977 | test_loss: 1.0321 | test_acc: 0.3210
 50% 5/10 [00:09<00:08,  1.67s/it]Epoch: 6 | train_loss: 0.9987 | train_acc: 0.4922 | test_loss: 1.0274 | test_acc: 0.3627
 60% 6/10 [00:11<00:06,  1.65s/it]Epoch: 7 | train_loss: 0.9169 | train_acc: 0.5781 | test_loss: 0.9627 | test_acc: 0.4536
 70% 7/10 [00:12<00:04,  1.56s/it]Epoch

In [13]:
!python going_modular/train_arg.py --num_epochs 3 --batch_size 64 --hidden_units 64 --learning_rate 0.002

[info] training a model for 3 epochs with a 64 batch_size using 64 hidden unist and a learning rate 0.002 
  0% 0/3 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1786 | train_acc: 0.3292 | test_loss: 1.1011 | test_acc: 0.2862
 33% 1/3 [00:03<00:06,  3.50s/it]Epoch: 2 | train_loss: 1.1005 | train_acc: 0.3216 | test_loss: 1.1755 | test_acc: 0.1953
 67% 2/3 [00:06<00:02,  2.94s/it]Epoch: 3 | train_loss: 1.0717 | train_acc: 0.3958 | test_loss: 0.9941 | test_acc: 0.6953
100% 3/3 [00:07<00:00,  2.51s/it]
[info] saving model to : models/05_going_modular_script_mode_tinyvgg_model.pth


In [ ]:
#create an one argument for train directory

In [14]:
%%writefile going_modular/train_arg.py
import os
import argparse
import torch 
from torchvision import transforms
import data_setup, engine, model_builder, utils

#create a parser
parser = argparse.ArgumentParser(description = "get some hyperparameters.")

#get an arg for num_epochs
parser.add_argument("--num_epochs",
                    default =10,
                    type=int,
                    help="the number of epochs to train for")
#get an arg for batch_size
parser.add_argument("--batch_size",
                    default = 32,
                    type = int,
                    help = "the number of sample per batch ")
#get an arg for hidden_units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help = " number of hidden units in hidden layers")
#get an arg for learning rate
parser.add_argument("--learning_rate",
                    default = 0.001,
                    type = float,
                    help= "learning rate to use for model")

#create an arg for training directory
parser.add_argument("--train_dir",
                    default = "data/pizza_steak_sushi/train",
                    type = str,
                    help = "directory file path to training data in standard image classification format")

#create an arg for test directory
parser.add_argument("--test_dir",
                    default = "data/pizza_steak_sushi/test",
                    type = str,
                    help = "directory file path to test data in standard image classificat")
#get out arguments from the parser
args = parser.parse_args()
#setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS =args.hidden_units
LEARNING_RATE = args.learning_rate
print(f"[info] training a model for {NUM_EPOCHS} epochs with a {BATCH_SIZE} batch_size using {HIDDEN_UNITS} hidden unist and a learning rate {LEARNING_RATE} ")
train_dir = args.train_dir
test_dir = args.test_dir
print(f"[info] training data file: {train_dir}")
print(f"[info] testing data file: {test_dir}")

device = "cuda" if torch.cuda.is_available() else "cpu"
data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir, 
    test_dir = test_dir,
    transform = data_transform,
    batch_size = BATCH_SIZE
)
model = model_builder.TinyVGG(
    input_shape =3,
    hidden_units = HIDDEN_UNITS,
    output_shape = len(class_names)
).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
engine.train(model = model,
             train_dataloader = train_dataloader,
             test_dataloader = test_dataloader,
             loss_fn = loss_fn,
             optimizer = optimizer,
             epochs = NUM_EPOCHS,
             device = device)
utils.save_model(model=model,
                 target_dir = "models",
                 model_name = "05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting going_modular/train_arg.py


In [15]:
!python going_modular/train_arg.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

[info] training a model for 5 epochs with a 128 batch_size using 128 hidden unist and a learning rate 0.0003 
[info] training data file: data/pizza_steak_sushi/train
[info] testing data file: data/pizza_steak_sushi/test
  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1036 | train_acc: 0.2967 | test_loss: 1.0976 | test_acc: 0.3333
 20% 1/5 [00:03<00:13,  3.27s/it]Epoch: 2 | train_loss: 1.0894 | train_acc: 0.3821 | test_loss: 1.0853 | test_acc: 0.4000
 40% 2/5 [00:04<00:06,  2.32s/it]Epoch: 3 | train_loss: 1.0681 | train_acc: 0.4884 | test_loss: 1.0573 | test_acc: 0.4400
 60% 3/5 [00:06<00:03,  2.00s/it]Epoch: 4 | train_loss: 1.0277 | train_acc: 0.5802 | test_loss: 1.0252 | test_acc: 0.4400
 80% 4/5 [00:08<00:01,  1.86s/it]Epoch: 5 | train_loss: 0.9611 | train_acc: 0.5129 | test_loss: 0.9985 | test_acc: 0.3600
100% 5/5 [00:10<00:00,  2.14s/it]
[info] saving model to : models/05_going_modular_script_mode_tinyvgg_model.pth


**Create a python script to predict (suchs as predict.py) on a target image given a file path with a saved model.**

* run the command python predict.py some_image.jpeg and have a trained pytorch model predict on the image and return its prediction.

In [16]:
%%writefile going_modular/predict.py
import torch
import torchvision
import argparse
import model_builder

#creating a parser
parser = argparse.ArgumentParser()
#get an image path
parser.add_argument("--image",
                    help="target image to predict on")

#get a model path
parser.add_argument("--model_path",
                    default = "models/05_going_modular_script_mode_tinyvgg_model.pth",
                    type = str,
                    help = "target model to use for prediction  filepath")

args = parser.parse_args()

#setup class names
class_names = ["pizza", "steak", "sushi"]
#setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

#get the image path
IMG_PATH = args.image
print(f"[info] predictiin on {IMG_PATH}")

#fucntion to load in the model
def load_model(filepath = args.model_path):
  #need to use same hyperparamerter as saved model
  model = model_builder.TinyVGG(input_shape =3,
                                hidden_units = 128,
                                output_shape =3).to(device)
  print(f"[info] loading in model from : {filepath}")
  #load in the saved model state dictionary from file
  model.load_state_dict(torch.load(filepath))
  
  return model

#fuction to laod in model + predict on select image
def predict_on_image(image_path = IMG_PATH, filepath=args.model_path):
  #load the model
  model = load_model(filepath)
  #load in the image and turn it into torch.float32 (same type as model)
  image = torchvision.io.read_image(str(IMG_PATH)).type(torch.float32)

  #preprocess the image
  image = image /255.
  #resise as same size in the model
  transform = torchvision.transforms.Resize((64,64))
  image = transform(image)
  image = image.unsqueeze(dim=0)

  #predcit on image
  model.eval()
  with torch.inference_mode():
    image = image.to(device)
    pred_logits = model(image)
    pred_prob = torch.softmax(pred_logits, dim=1)
    pred_label = torch.argmax(pred_prob, dim=1)
    pred_label_class = class_names[pred_label]
  print(f"[info] pred class: {pred_label_class}, pred prob: {pred_prob.max():.3f}")

if __name__ == "__main__":
  predict_on_image()

Writing going_modular/predict.py


In [17]:
!python going_modular/predict.py --image data/pizza_steak_sushi/test/sushi/1172255.jpg

[info] predictiin on data/pizza_steak_sushi/test/sushi/1172255.jpg
[info] loading in model from : models/05_going_modular_script_mode_tinyvgg_model.pth
[info] pred class: pizza, pred prob: 0.469


In [ ]:
#target is
!python predict.py some_image.jpeg

In [ ]:
/content/data